In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
# import all the data

df = pd.read_csv("../data/data.csv")

In [ ]:
df.head(10)

In [ ]:
#to get which year the player is in and create Year column

i = 0
while (i < len(df)):
    if(df.loc[i, 'NBA'] == 0 or df.loc[i, 'Season'] == 'Career'):
        df.loc[i, 'Year'] = 0
    else:
        j = i
        yrno = 1
        while(df.loc[j, 'Name'] == df.loc[i, 'Name'] and df.loc[j, 'NBA'] == 1 and df.loc[j, 'Season'] != 'Career'):
            df.loc[j, 'Year'] = yrno
            if(df.loc[j, 'Season'] == df.loc[j + 1, 'Season']):
                yrno -= 1
            yrno += 1
            j += 1
        i = j - 1
    i += 1

In [ ]:
#join stats for players who got traded in between season

i = 0
while (i < len(df)-1):
    if(df.loc[i, 'Season'] == df.loc[i+1, 'Season'] and df.loc[i, 'Name'] == df.loc[i+1, 'Name']):
        df.loc[i, 'Team'] = df.loc[i, 'Team'] + "/" + df.loc[i+1, 'Team']
        df.loc[i, 'MIN'] = (df.loc[i, 'MIN']*df.loc[i, 'GP'] + df.loc[i+1, 'MIN']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'FG'] = (df.loc[i, 'FG']*df.loc[i, 'GP'] + df.loc[i+1, 'FG']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'FGA'] = (df.loc[i, 'FGA']*df.loc[i, 'GP'] + df.loc[i+1, 'FGA']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, '3PT'] = (df.loc[i, '3PT']*df.loc[i, 'GP'] + df.loc[i+1, '3PT']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, '3PTA'] = (df.loc[i, '3PTA']*df.loc[i, 'GP'] + df.loc[i+1, '3PTA']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'FT'] = (df.loc[i, 'FT']*df.loc[i, 'GP'] + df.loc[i+1, 'FT']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'FTA'] = (df.loc[i, 'FTA']*df.loc[i, 'GP'] + df.loc[i+1, 'FTA']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'OR'] = (df.loc[i, 'OR']*df.loc[i, 'GP'] + df.loc[i+1, 'OR']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'DR'] = (df.loc[i, 'DR']*df.loc[i, 'GP'] + df.loc[i+1, 'DR']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'AST'] = (df.loc[i, 'AST']*df.loc[i, 'GP'] + df.loc[i+1, 'AST']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'BLK'] = (df.loc[i, 'BLK']*df.loc[i, 'GP'] + df.loc[i+1, 'BLK']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'STL'] = (df.loc[i, 'STL']*df.loc[i, 'GP'] + df.loc[i+1, 'STL']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'PF'] = (df.loc[i, 'PF']*df.loc[i, 'GP'] + df.loc[i+1, 'PF']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'PTS'] = (df.loc[i, 'PTS']*df.loc[i, 'GP'] + df.loc[i+1, 'PTS']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'TO'] = (df.loc[i, 'TO']*df.loc[i, 'GP'] + df.loc[i+1, 'TO']*df.loc[i+1, 'GP'])/(df.loc[i, 'GP'] + df.loc[i+1, 'GP'])
        df.loc[i, 'GP'] = df.loc[i, 'GP'] + df.loc[i+1, 'GP']
        df.loc[i, 'GS'] = df.loc[i, 'GS'] + df.loc[i+1, 'GS']
        if(df.loc[i, 'FGA'] != 0):
            df.loc[i, 'FG%'] = df.loc[i, 'FG']/df.loc[i, 'FGA'] * 100
        else: 
            df.loc[i, 'FG%'] = 0
        if(df.loc[i, '3PTA'] != 0):
            df.loc[i, '3P%'] = df.loc[i, '3PT']/df.loc[i, '3PTA'] * 100
        else:
            df.loc[i, '3P%'] = 0
        if(df.loc[i, 'FTA'] != 0):
            df.loc[i, 'FT%'] = df.loc[i, 'FT']/df.loc[i, 'FTA'] * 100
        else:
            df.loc[i, 'FT%'] = 0
        df.loc[i, 'REB'] = df.loc[i, 'OR'] + df.loc[i+1, 'DR']
        df = df.drop(i + 1)
        i += 1
    i += 1

In [ ]:
# Ignore cells below

# These cells use all players data and not only players who went to college

In [ ]:
df_temp = df.copy()

In [ ]:
columns = ['GP', 'GS', 'MIN', 'FG', 'FGA', '3PT', '3PTA', 'FT', 'FTA', 'REB', 'AST', 'BLK', 'STL', 'PF', 'TO', 'PTS']

In [ ]:
for column in columns:
    for yr in range(1, 19):
        maxval = df_temp[df_temp['Year'] == yr][column].max()
        minval = df_temp[df_temp['Year'] == yr][column].min()

        df_temp.loc[df.Year == yr, column] = (df_temp.loc[df.Year == yr, column] - minval)/(maxval - minval)

In [ ]:
df_temp = df_temp[(df_temp['NBA'] == 1) & (df_temp['Season'] != 'Career')]

In [ ]:
df_temp["Score"] = df_temp["GP"] + 0.5*df_temp["GS"] + df_temp["MIN"] + df_temp["FG"] + 0.75*df_temp["FGA"] + df_temp["FT"] + 0.75*df_temp["FTA"] + 1.5*df_temp["REB"] + 1.5*df_temp["AST"] + df_temp["BLK"] + df_temp["STL"] - 0.5*df_temp["PF"] - df_temp["TO"] + 1.5*df_temp["PTS"]

In [ ]:
# Best seasons in the league

df_temp.nlargest(20, ['Score'])

In [ ]:
# Dictionary with all players from college

players = {}
count = 0

for name in nba_df['Name'].unique():
    players[count] = name
    count += 1

print(count)

In [ ]:
# Stop ignoring

In [ ]:
# number of nba players from college

len(df[df['NBA'] == 0]['Name'].unique())

In [ ]:
# get only college players in the league and put in use_df

use_df = df.iloc[0:0,:].copy()
for name in df[df['NBA'] == 0]['Name'].unique():
    use_df = pd.concat([use_df, df[df['Name'] == name]])

In [ ]:
# example of player from college

use_df[use_df["Name"] == "Carmelo Anthony"]

In [ ]:
# normalize every stat with maximum/minimum of each year in the league

for column in columns:
    for yr in range(1, 19):
        maxval = use_df[use_df['Year'] == yr][column].max()
        minval = use_df[use_df['Year'] == yr][column].min()

        use_df.loc[use_df.Year == yr, column] = (use_df.loc[use_df.Year == yr, column] - minval)/(maxval - minval)

In [ ]:
# get yearly stats from the NBA

nba_df = use_df[(use_df['NBA'] == 1) & (use_df['Season'] != 'Career')]

In [ ]:
# create new column to get a score for each season
# takes different weights for each stat based on importance I've given to them
# can be changed to make more sense 

nba_df["Score"] = nba_df["GP"] + 0.5*nba_df["GS"] + nba_df["MIN"] + nba_df["FG"] + 0.75*nba_df["FGA"] + nba_df["FT"] + 0.75*nba_df["FTA"] + 1.5*nba_df["REB"] + 1.5*nba_df["AST"] + nba_df["BLK"] + nba_df["STL"] - 0.5*nba_df["PF"] - nba_df["TO"] + 1.5*nba_df["PTS"]

In [ ]:
# get best players from each season of career 

for yr in range(1, 19):
    print(yr, nba_df[nba_df['Score'] == nba_df[nba_df["Year"] == yr]['Score'].max()]["Name"])

In [ ]:
# create a dictionary to have a list of all scores in career 

careers = {}
for name in nba_df['Name'].unique():
    temp = nba_df[nba_df['Name'] == name]['Score'].tolist()
    careers[name] = temp
    
careers

In [ ]:
# to get average size of career of players in dataset

yrs = np.zeros(427)

i = 0
for key in careers:
    yrs[i] = len(careers[key])
    i += 1
    
yrs.mean()

In [ ]:
# create a dataframe with scores from each season

k_df = pd.DataFrame(columns=["name", "yr1", "yr2", "yr3", "yr4", "yr5"])


for key in careers:
    if(len(careers[key]) > 4):
        k_df = k_df.append({"name": key, "yr1": careers[key][0], "yr2": careers[key][1], "yr3": careers[key][2], "yr4": careers[key][3], "yr5": careers[key][4]}, ignore_index=True)
        
k_df.head(20)

In [ ]:
# assign a cluster to each player in the dataframe using KMeans

model = KMeans(n_clusters = 8)
features = k_df[['yr1', 'yr2', 'yr3', 'yr4', 'yr5']]
fit = model.fit(features)
k_df['cluster'] = model.predict(features)
k_df.head(20)

In [ ]:
# example of players in a cluster

k_df[k_df['cluster'] == 1]

In [ ]:
# visualise the careers of players from each cluster 

plt.figure(figsize=(16, 8)) 

for i in range(1, 9):
    plt.subplot(2, 4, i)
    temp = k_df[k_df['cluster'] == i - 1][['yr1', 'yr2', 'yr3', 'yr4', 'yr5']]
    plt.plot(temp.T)
    plt.tight_layout()
    plt.xticks(fontsize = 20)
    plt.ylim([0, 10])
    plt.yticks(fontsize = 20)
    plt.title("Cluster " + str(i), fontsize = 25)
    plt.ylabel("Score", fontsize = 20)